In [20]:
import pandas as pd

In [21]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Deepak@121",
    database="videogamesdb"
)
cursor = conn.cursor()


In [22]:
cursor.execute("CREATE DATABASE IF NOT EXISTS VideoGamesdb")
cursor.execute("USE VideoGamesdb")


In [25]:
# Create Games Table

cursor.execute("""
CREATE TABLE games (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Title VARCHAR(255),
    Release_Date DATE,
    Team VARCHAR(255),
    Rating FLOAT,
    Times_Listed INT,
    Genres VARCHAR(255),
    Summary TEXT,
    Reviews TEXT,
    Plays INT,
    Playing INT,
    Backlogs INT,
    Wishlist INT
);
""")


In [26]:
games = pd.read_csv("games_clean.csv")

In [27]:
# Missing values handle
games['Summary'] = games['Summary'].fillna("No Summary")
games['Reviews'] = games['Reviews'].fillna("No Reviews")


In [28]:
# Load cleaned CSV
games = pd.read_csv("games_clean.csv")

# Fill NaNs
games['Summary'] = games['Summary'].fillna("No Summary Available")
games['Reviews'] = games['Reviews'].fillna("No Reviews")

# Insert into SQL
for _, row in games.iterrows():
    cursor.execute ("""
        INSERT INTO games 
        (Title, Release_Date, Team, Rating, Times_Listed, Genres, Summary, Reviews, Plays, Playing, Backlogs, Wishlist)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Title'], 
        row['Release_Date'], 
        row['Team'], 
        row['Rating'], 
        row['Times_Listed'], 
        row['Genres'], 
        row['Summary'], 
        row['Reviews'], 
        row['Plays'], 
        row['Playing'], 
        row['Backlogs'], 
        row['Wishlist']
    ))

conn.commit()


In [29]:
sales = pd.read_csv("sales_clean.csv")

In [32]:
sales.head(1)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,clean_title
0,1,Wii Sports,Wii,NaN,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,wii sports


In [33]:
#Create sales table 

cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    `Rank` INT,
    Name VARCHAR(255),
    Platform VARCHAR(100),
    Year DATE,
    Genre VARCHAR(100),
    Publisher VARCHAR(255),
    NA_Sales FLOAT,
    EU_Sales FLOAT,
    JP_Sales FLOAT,
    Other_Sales FLOAT,
    Global_Sales FLOAT
)
""")

In [ ]:
# Year format in YYYY-01-01
sales['Year'] = pd.to_datetime(sales['Year'], errors='coerce').fillna(pd.Timestamp("1900-01-01"))


In [42]:
for _, row in sales.iterrows():
    cursor.execute("""
        INSERT INTO sales
        (`Rank`, Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Rank'],
        row['Name'],
        row['Platform'],
        row['Year'].strftime('%Y-%m-%d'),
        row['Genre'],
        row['Publisher'],
        row['NA_Sales'],
        row['EU_Sales'],
        row['JP_Sales'],
        row['Other_Sales'],
        row['Global_Sales']
    ))
    
conn.commit()
print("Sales data inserted successfully!")

Sales data inserted successfully!


In [35]:
# Create Merged Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS merged_games_sales (
    Title VARCHAR(255),
    Release_Date DATE,
    Team VARCHAR(255),
    Rating FLOAT,
    Times_Listed INT,
    Genres VARCHAR(255),
    Summary TEXT,
    Reviews TEXT,
    Plays INT,
    Playing INT,
    Backlogs INT,
    Wishlist INT,
    `Rank` INT,
    Platform VARCHAR(100),
    Year DATE,
    Genre VARCHAR(100),
    Publisher VARCHAR(255),
    NA_Sales FLOAT,
    EU_Sales FLOAT,
    JP_Sales FLOAT,
    Other_Sales FLOAT,
    Global_Sales FLOAT
)
""")
print("Merged table created successfully!")

Merged table created successfully!


In [39]:
merged = pd.read_csv("merged_games_sales.csv")

In [41]:
# Dates format in YYYY-MM-DD
merged['Release_Date'] = pd.to_datetime(merged['Release_Date'], errors='coerce').fillna(pd.Timestamp("1900-01-01"))
merged['Year'] = pd.to_datetime(merged['Year'], errors='coerce').fillna(pd.Timestamp("1900-01-01"))

# Insert merged data
for _, row in merged.iterrows():
    cursor.execute("""
        INSERT INTO merged_games_sales
        (Title, Release_Date, Team, Rating, Times_Listed, Genres, Summary, Reviews, Plays, Playing, Backlogs, Wishlist,
        `Rank`, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Title'],
        row['Release_Date'].strftime('%Y-%m-%d'),
        row['Team'],
        row['Rating'],
        row['Times_Listed'],
        row['Genres'],
        row['Summary'],
        row['Reviews'],
        row['Plays'],
        row['Playing'],
        row['Backlogs'],
        row['Wishlist'],
        row['Rank'],
        row['Platform'],
        row['Year'].strftime('%Y-%m-%d'),
        row['Genre'],
        row['Publisher'],
        row['NA_Sales'],
        row['EU_Sales'],
        row['JP_Sales'],
        row['Other_Sales'],
        row['Global_Sales']
    ))

conn.commit()
print("Merged data inserted successfully!")

Merged data inserted successfully!
